In [2]:
import pandas as pd
import numpy as np
from os import path
import sys
import tensorflow as tf

#plotting
from matplotlib import pyplot as plt

#processing
from scipy.signal import resample


In [1]:
import pandas_profiling as pdpf


In [3]:
features=['x','y','pen_up','pressure']
cols=[0, 1, 3, 6]
doc_path = "/data/elekin/doc"
meta_df=pd.read_csv(path.join(doc_path,"metadata-202106-v1.csv"),index_col=0)
x_train = []
y_train = []

In [6]:
pdpf.ProfileReport(meta_df).to_file(path.join(doc_path,"spiral-meta-profiling.html"))


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1541: FutureWarning: split() requires a non-empty pattern match.
  f = lambda x: regex.split(x, maxsplit=n)
/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1541: FutureWarning: split() requires a non-empty pattern match.
  f = lambda x: regex.split(x, maxsplit=n)
/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1541: FutureWarning: split() requires a non-empty pattern match.
  f = lambda x: regex.split(x, maxsplit=n)
/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1541: FutureWarning: split() requires a non-empty pattern match.
  f = lambda x: regex.split(x, maxsplit=n)
/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1541: FutureWarning: split() requires a non-empty pattern match.
  f = lambda x: regex.split(x, maxsplit=n)
/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1541: FutureWarning: split() requires a non-empty pattern match.
  f = lambda x: regex.s

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Creating a data structure with subjects time series and tremor levels
 
X = []
y = []

for path, level in zip(meta_df.abs_path, meta_df.level):
    df = pd.read_csv(path, sep="\s+",header=None,names=features,skiprows=1,usecols=[0,1,3,6])
    X.append(resample(df.values.astype('int16'), 4096))
    y.append(level)

In [16]:
with open('feat.npy', 'wb') as temp_feat:
    np.save(temp_feat, np.array(X, dtype=np.float32), allow_pickle=True)
with open('labels.npy', 'wb') as temp_labels:
    np.save(temp_labels, np.array(y, dtype=np.int8), allow_pickle=True)

In [17]:
with open('labels.npy', 'rb') as temp_labels:
    y_train = np.load(temp_labels, allow_pickle=True)
with open('feat.npy', 'rb') as temp_feat:
    X_train = np.load(temp_feat, allow_pickle=True)

In [8]:
X_train.shape

(53, 4096, 4)

In [9]:
y_train.shape

(53,)

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, tf.one_hot(y_train, 3)))

In [11]:
tf.data.experimental.save(dataset, "./ethw.tfdata")

In [12]:
len(dataset)

53

In [15]:
for feat, label in tf.data.experimental.load("./ethw.tfdata").take(1):
    print(feat.shape)
    

(4096, 4)
